In [1]:
import sys
sys.path.append('../../')
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from model import GTN
import pdb
import pickle
import argparse
from utils import f1_score
from scripts.data_loader import data_loader
import scipy.sparse as sp
from augment import Augmenter
from tools import evaluate_results_nc


def consis_loss(logps, temp):
    ps = [torch.exp(p) for p in logps]
    sum_p = 0.
    for p in ps:
        sum_p = sum_p + p
    avg_p = sum_p/len(ps)
    #p2 = torch.exp(logp2)
    
    sharp_p = (torch.pow(avg_p, 1./temp) / torch.sum(torch.pow(avg_p, 1./temp), dim=1, keepdim=True)).detach()
    loss = 0.
    for p in ps:
        loss += torch.mean((p-sharp_p).pow(2).sum(1))
    loss = loss/len(ps)
    return args.rate * loss


def load_data(args):
    dataset, full, feats_type = args.dataset, args.full, args.feats_type
    drop_feat = []
    if dataset == 'DBLP':
        if feats_type == 1:
            drop_feat = [0,1,3]
        elif feats_type == 2:
            drop_feat = [0]
    dl = data_loader('/ossfs/workspace/HGB/NC/benchmark/methods/GTN/'+dataset)
    if dataset == 'DBLP' and not full:
        dl.get_sub_graph([0,1,3])
    if dataset == 'ACM' and not full:
        dl.get_sub_graph([0,1,2])
    edges = list(dl.links['data'].values())
    node_features = []
    w_ins = []
    for k in dl.nodes['attr']:
        th = dl.nodes['attr'][k][:,:64]
        if th is None or k in drop_feat:
            cnt = dl.nodes['count'][k]
            node_features.append(sp.eye(cnt))
            w_ins.append(cnt)
        else:
            node_features.append(th)
            w_ins.append(th.shape[1])
    val_ratio = 0.2
    train_idx = np.nonzero(dl.labels_train['mask'])[0]
    np.random.shuffle(train_idx)
    split = int(train_idx.shape[0]*val_ratio)
    val_idx = train_idx[:split]
    train_idx = train_idx[split:]
    train_idx = np.sort(train_idx)
    val_idx = np.sort(val_idx)
    test_idx = np.nonzero(dl.labels_test['mask'])[0]
    labels = np.zeros((dl.nodes['count'][0], dl.labels_train['num_classes']), dtype=int)
    labels[train_idx] = dl.labels_train['data'][train_idx]
    labels[val_idx] = dl.labels_train['data'][val_idx]
    labels = labels.argmax(axis=1)
    train_label = labels[train_idx]
    val_label = labels[val_idx]
    train_label = list(zip(train_idx.tolist(), train_label.tolist()))
    val_label = list(zip(val_idx.tolist(), val_label.tolist()))
    labels = [train_label, val_label, test_idx.tolist()]
    return node_features, edges, labels, dl, w_ins

def sp_to_spt(mat):
    coo = mat.tocoo()
    values = coo.data
    indices = np.vstack((coo.row, coo.col))

    i = torch.LongTensor(indices)
    v = torch.FloatTensor(values)
    shape = coo.shape

    return torch.sparse.FloatTensor(i, v, torch.Size(shape))

def mat2tensor(mat):
    if type(mat) is np.ndarray:
        return torch.from_numpy(mat).type(torch.FloatTensor)
    return sp_to_spt(mat)

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--dataset', type=str, default='JD2',
                        help='DATASET')
    parser.add_argument('--epoch', type=int, default=40,
                        help='Training Epochs')
    parser.add_argument('--node_dim', type=int, default=64,
                        help='Node dimension')
    parser.add_argument('--num_channels', type=int, default=2,
                        help='number of channels')
    parser.add_argument('--lr', type=float, default=0.1,
                        help='learning rate')
    parser.add_argument('--weight_decay', type=float, default=1e-3,
                        help='l2 reg')
    parser.add_argument('--num_layers', type=int, default=2,
                        help='number of layer')
    parser.add_argument('--norm', type=str, default='true',
                        help='normalization')
    parser.add_argument('--adaptive_lr', type=str, default='false',
                        help='adaptive learning rate')
    parser.add_argument('--full', type=bool, default=False)
    parser.add_argument('--feats_type', type=int, default=0)
    parser.add_argument('--device', type=int, default=0)
    parser.add_argument('--cf_k', type=int, default=2)
    parser.add_argument('--tem', type=float, default=0.2)
    parser.add_argument('--rate', type=float, default=0.2)
    args = parser.parse_args(args=[])
    print(args)
    epochs = args.epoch
    node_dim = args.node_dim
    num_channels = args.num_channels
    lr = args.lr
    weight_decay = args.weight_decay
    num_layers = args.num_layers
    norm = args.norm
    adaptive_lr = args.adaptive_lr
    
    if args.device >= 0:
        device = torch.device("cuda:"+str(args.device))
    else:
        device = torch.device('cpu')

    """with open('data/'+args.dataset+'/node_features.pkl','rb') as f:
        node_features = pickle.load(f)
    with open('data/'+args.dataset+'/edges.pkl','rb') as f:
        edges = pickle.load(f)
    with open('data/'+args.dataset+'/labels.pkl','rb') as f:
        labels = pickle.load(f)"""
    node_features, edges, labels, dl, w_ins = load_data(args)
    
    num_nodes = edges[0].shape[0]
    for i,edge in enumerate(edges):
        if i ==0:
            A = torch.from_numpy(edge.todense()).type(torch.FloatTensor).unsqueeze(-1)
        else:
            A = torch.cat([A,torch.from_numpy(edge.todense()).type(torch.FloatTensor).unsqueeze(-1)], dim=-1)
    A = torch.cat([A,torch.eye(num_nodes).type(torch.FloatTensor).unsqueeze(-1)], dim=-1).to(device)
    A[torch.isnan(A)]=0.01
#     print('A>>>>>>>>>',torch.any(torch.isnan(A)))
    
    node_features = [mat2tensor(x) for x in node_features] # torch.from_numpy(node_features).type(torch.FloatTensor)
#     print('node_features>>>>',type(node_features),len(node_features[1]),len(node_features[1][0]))
    train_node = torch.from_numpy(np.array(labels[0])[:,0]).type(torch.LongTensor).to(device)
    train_target = torch.from_numpy(np.array(labels[0])[:,1]).type(torch.LongTensor).to(device)
    valid_node = torch.from_numpy(np.array(labels[1])[:,0]).type(torch.LongTensor).to(device)
    valid_target = torch.from_numpy(np.array(labels[1])[:,1]).type(torch.LongTensor).to(device)
    test_node = torch.from_numpy(np.array(labels[2])).type(torch.LongTensor).to(device)
    unlabel_idx = test_node.cpu()
    
    num_classes = torch.max(train_target).item()+1
    final_f1 = 0
    for l in range(1):
        model = GTN(num_edge=A.shape[-1],
                            num_channels=num_channels,
                            w_ins = w_ins, #node_features.shape[1],
                            w_out = node_dim,
                            num_class=num_classes,
                            num_layers=num_layers,
                            norm=norm).to(device)
        if adaptive_lr == 'false':
            optimizer = torch.optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)
        else:
            optimizer = torch.optim.Adam([{'params':model.weights},
                                        {'params':model.linear1.parameters()},
                                        {'params':model.linear2.parameters()},
                                        {"params":model.layers.parameters(), "lr":0.05}
                                        ], lr=0.005, weight_decay=args.weight_decay)
        loss = nn.CrossEntropyLoss().to(device)
        # Train & Valid & Test
        best_val_loss = 10000
        best_test_loss = 10000
        best_train_loss = 10000
        best_train_f1 = 0
        best_val_f1 = 0
        best_test_f1 = 0
        test_embeddings = []
        
        augmenter = Augmenter(args=args, features_list=node_features, is_label=labels)
        
    
        for i in range(epochs):
            loss_u_list = []
            
            for param_group in optimizer.param_groups:
                if param_group['lr'] > 0.005:
                    param_group['lr'] = param_group['lr'] * 0.9
            print('Epoch:  ',i+1)
            model.zero_grad()
            model.train()
            
            # 正向传播时：开启自动求导的异常侦测
#             torch.autograd.set_detect_anomaly(True)
            
            loss_l,y_train,Ws,embed,all_y = model(A, node_features, train_node, train_target)
            loss_u_list.append(torch.log_softmax(all_y[unlabel_idx], dim=-1))
#             ------ 节点增强 ------
            aug_features = augmenter.node_aug_heterograph_jd(node_features, unlabel_idx, args.cf_k)
            _,y_train_u1,Ws_u1,embed1,all_y1 = model(A, aug_features, train_node, train_target)
            loss_u_list.append(torch.log_softmax(all_y1[unlabel_idx], dim=-1))
#             # ------ 结构增强 ------
            aug_A = augmenter.triangle_aug_heterograph()
            _,y_train_u2,Ws_u2,embed2,all_y2 = model(aug_A, node_features, train_node, train_target)
            loss_u_list.append(torch.log_softmax(all_y2[unlabel_idx], dim=-1))
            
            loss_u = consis_loss(loss_u_list, temp=args.tem)
            loss = loss_l + loss_u
            
            train_f1 = torch.mean(f1_score(torch.argmax(y_train.detach(),dim=1), train_target, num_classes=num_classes)).cpu().numpy()
            print('Train - Loss: {}, Macro_F1: {}'.format(loss.detach().cpu().numpy(), train_f1))
            
#             for name, parms in model.named_parameters():
#                 print('-->name:', name, '-->grad_requirs:', parms.requires_grad)
#                 print('--weight', torch.mean(parms.data))
#                 print('-->grad_value:', torch.mean(parms.grad)) 
            # 反向传播时：在求导时开启侦测
#             with torch.autograd.detect_anomaly():
            loss.backward()
    
            optimizer.step()
            model.eval()
            # Valid
            with torch.no_grad():
#                 print('A 2 2>>>>>>>>>',torch.any(torch.isnan(A)))
                val_loss,y_valid,_,_,all_y_valid = model(A, node_features, valid_node, valid_target)
                val_f1 = torch.mean(f1_score(torch.argmax(y_valid,dim=1), valid_target, num_classes=num_classes)).cpu().numpy()
                print('Valid - Loss: {}, Macro_F1: {}'.format(val_loss.detach().cpu().numpy(), val_f1))
                test_loss,y_test,W,test_emb,_ = model(A, node_features, test_node, None)
                pred = y_test.cpu().numpy().argmax(axis=1)
                onehot = np.eye(num_classes, dtype=np.int32)
                pred = onehot[pred]
                test_f1 = dl.evaluate(pred)
                print('test_f1>>>>>> ',test_f1)
                
                #test_f1 = torch.mean(f1_score(torch.argmax(y_test,dim=1), test_target, num_classes=num_classes)).cpu().numpy()
                #print('Test - Loss: {}, Macro_F1: {}\n'.format(test_loss.detach().cpu().numpy(), test_f1))
            if val_f1 > best_val_f1:
                best_val_loss = val_loss.detach().cpu().numpy()
                #best_test_loss = test_loss.detach().cpu().numpy()
                best_train_loss = loss.detach().cpu().numpy()
                best_train_f1 = train_f1
                best_val_f1 = val_f1
                best_test_f1 = test_f1 
                test_embeddings = test_emb
        print('---------------Best Results--------------------')
        print('Train - Loss: {}, Macro_F1: {}'.format(best_train_loss, best_train_f1))
        print('Valid - Loss: {}, Macro_F1: {}'.format(best_val_loss, best_val_f1))
        print('best_test_f1>>>>>> ',best_test_f1)
        svm_macro_f1_list, svm_micro_f1_list, nmi_mean, nmi_std, ari_mean, ari_std = evaluate_results_nc(
                test_embeddings[test_node].cpu().numpy(), dl.labels_test['data'][dl.labels_test['mask']].argmax(axis=1), num_classes=dl.labels_test['num_classes'])

#         print('Test - Loss: {}, Macro_F1: {}'.format(best_test_loss, best_test_f1))
#         final_f1 += best_test_f1


/opt/conda/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using backend: pytorch


Namespace(adaptive_lr='false', cf_k=2, dataset='JD2', device=0, epoch=40, feats_type=0, full=False, lr=0.1, node_dim=64, norm='true', num_channels=2, num_layers=2, rate=0.2, tem=0.2, weight_decay=0.001)
loading data:
loading data >>>>>>>>>>>>>>>>>>>>>
0 {'0': 3587}
1 {'0': 3587, '1': 5916}
building dgl.heterograph >>>>>>>>>>>>>>>>>>>>>
sample_num: 5370
Epoch:   1


/ossfs/workspace/HGB/NC/benchmark/methods/GTN/augment.py:338: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  neigh_fes = torch.tensor(neigh_fes/2.0, dtype=torch.float)


aug heterograph time: 7.099057674407959
Train - Loss: 10.53352165222168, Macro_F1: 0.3753623068332672
Valid - Loss: 130703.640625, Macro_F1: 0.2973856031894684
test_f1>>>>>>  {'micro-f1': 0.40199203187251, 'macro-f1': 0.2867291844273942}
Epoch:   2


/ossfs/workspace/HGB/NC/benchmark/methods/GTN/augment.py:338: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  neigh_fes = torch.tensor(neigh_fes/2.0, dtype=torch.float)


...

aug heterograph time: 6.9866650104522705
Train - Loss: 3.1341817378997803, Macro_F1: 0.3753623068332672
Valid - Loss: 2.3880507946014404, Macro_F1: 0.3657817244529724
test_f1>>>>>>  {'micro-f1': 0.59800796812749, 'macro-f1': 0.37422089254549984}
Epoch:   39


/ossfs/workspace/HGB/NC/benchmark/methods/GTN/augment.py:338: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  neigh_fes = torch.tensor(neigh_fes/2.0, dtype=torch.float)


aug heterograph time: 6.849695444107056
Train - Loss: 2.4510276317596436, Macro_F1: 0.3753623068332672
Valid - Loss: 1.805169701576233, Macro_F1: 0.3657817244529724
test_f1>>>>>>  {'micro-f1': 0.59800796812749, 'macro-f1': 0.37422089254549984}
Epoch:   40


/ossfs/workspace/HGB/NC/benchmark/methods/GTN/augment.py:338: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  neigh_fes = torch.tensor(neigh_fes/2.0, dtype=torch.float)


aug heterograph time: 6.823050498962402
Train - Loss: 1.9080349206924438, Macro_F1: 0.3753623068332672
Valid - Loss: 1.3752201795578003, Macro_F1: 0.3657817244529724
test_f1>>>>>>  {'micro-f1': 0.59800796812749, 'macro-f1': 0.37422089254549984}
---------------Best Results--------------------
Train - Loss: 133712.09375, Macro_F1: 0.2852404713630676
Valid - Loss: 6672.380859375, Macro_F1: 0.3657817244529724
best_test_f1>>>>>>  {'micro-f1': 0.59800796812749, 'macro-f1': 0.37422089254549984}
SVM test
Macro-F1: 0.750018~0.019287 (0.8), 0.747448~0.009412 (0.6), 0.748067~0.009937 (0.4), 0.745735~0.010568 (0.2)
Micro-F1: 0.765737~0.016065 (0.8), 0.762649~0.009259 (0.6), 0.762749~0.008794 (0.4), 0.760408~0.010405 (0.2)
K-means test
NMI: 0.158555~0.000000
ARI: 0.220237~0.000000


In [1]:
1

1